In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib.request
import sys
import torch
import time
import datetime

from torchvision import transforms
from PIL import Image
from moviepy.editor import *

%matplotlib inline

In [ ]:
sys.version

In [ ]:
print(datetime.datetime.today())

In [ ]:
!nvidia-smi